In [1]:
"conda_amazonei_tensorflow_p36"

In [1]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import numpy as np
import boto3

In [2]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")


In [3]:
print(role)
print(bucket)

arn:aws:iam::535328050074:role/AWSGlueServiceSageMakerNotebookRole
sagemaker-us-west-2-535328050074


In [4]:
model_package_arn = "arn:aws:sagemaker:us-west-2:535328050074:model-package/ct-scan-body-part-detector-v1"

model_name = "ct-scan-body-part-detector"

content_type = "application/json"

real_time_inference_instance_type = 'ml.m5.xlarge'
batch_transform_inference_instance_type = 'ml.m5.xlarge'

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------------!

In [12]:
! pip install SimpleITK
! pip install pydicom

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 48.4 MB 7.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/bin/python -m pip install --upgrade pip' command.
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/home/ec2-user/anaconda3/envs/amazonei_tensor

In [45]:
import os
import requests
from base64 import b64encode,b64decode
import json
import zipfile
import numpy as np
import pydicom
import SimpleITK as sitk
import matplotlib.pyplot as plt

def resample_img(itk_image, out_spacing):
    
    # Resample images to out_spacing with SimpleITK
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())
    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

# custom resample
def imread(myinput,downsample=True):
    if isinstance(myinput,list):
        file_list = myinput
        dicom_list = []
        for image_file in file_list:
            ds=pydicom.dcmread(image_file)
            dicom_list.append((ds.InstanceNumber,image_file))
            dicom_list = sorted(dicom_list,key=lambda x:x[0])
        dicom_names = [x[1] for x in dicom_list]
        reader = sitk.ImageSeriesReader()
        reader.SetFileNames(dicom_names)
    else:
        file_path = myinput
        reader= sitk.ImageFileReader()
        reader.SetFileName(file_path)
    img_obj = reader.Execute()
    if downsample:
        spacing = list(img_obj.GetSpacing())
        size = list(img_obj.GetSize())
        
        # scale down to 128x128 for dimensions 0 and 1
        spacing[0] = spacing[0]/128.
        spacing[1] = spacing[1]/128.
        
        # scale to no more than 50 scans (for dimension 2)
        desired_z = 50.0
        if size[2] < desired_z:
            factor = 1
        else:
            factor = size[2]/desired_z
        spacing[2] = spacing[2]*factor
        
        img_obj = resample_img(img_obj, out_spacing=spacing)
    return img_obj

def imwrite(fpath,img_obj,use_compression=True):
    writer = sitk.ImageFileWriter()    
    writer.SetFileName(fpath)
    writer.SetUseCompression(use_compression)
    writer.Execute(img_obj)

def download_images(series_instance_uid,image_root="."):
    url = f"https://services.cancerimagingarchive.net/services/v4/TCIA/query/getImage?SeriesInstanceUID={series_instance_uid}"
    zip_file_path = os.path.join(image_root,series_instance_uid+'.zip')
    r = requests.get(url, allow_redirects=True)
    print(r.status_code)
    if r.status_code != 200:
        raise LookupError(f"ohoh {r.status_code}!")
    open(zip_file_path, 'wb').write(r.content)
    folder_path = os.path.join(image_root,series_instance_uid)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(folder_path)
    return folder_path

def convert_to_nifti(folder_path,series_instance_uid,image_root=None):
    file_list = [os.path.join(folder_path,x) for x in os.listdir(folder_path)]
    img_obj = imread(file_list)
    nii_gz_path = os.path.join(image_root,f'{series_instance_uid}.nii.gz')
    imwrite(nii_gz_path,img_obj)
    return nii_gz_path

ENCODING = 'utf-8'
def convert_to_json(input_file):
    with open(input_file, 'rb') as open_file:
        byte_content = open_file.read()
    base64_bytes = b64encode(byte_content)
    base64_string = base64_bytes.decode(ENCODING)
    myjson = {"niigz":base64_string}
    return myjson

def upload_to_s3(input_file):
    s3_prefix = 'ct-sample-images'
    rawdata_s3_prefix = '{}'.format(s3_prefix)
    s3_uri = sess.upload_data(path=input_file, key_prefix=rawdata_s3_prefix)
    return s3_uri
     

In [48]:

series_instance_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.113679818447732724990336702075'
image_root = "myimages"
os.makedirs(image_root,exist_ok=True)
# download images from TCIA
folder_path = download_images(series_instance_uid,image_root=image_root)
nii_gz_path = convert_to_nifti(folder_path,series_instance_uid,image_root=image_root)
mypayload = convert_to_json(nii_gz_path)

In [50]:
with open('mypayload.txt','w') as f:
    f.write(json.dumps(mypayload))

In [79]:
with open('mypayload.txt','r') as f:
    mypayloadstr=f.read()

In [80]:
!ls -lh mypayload.txt

-rw-rw-r-- 1 ec2-user ec2-user 1.4M Sep 12 21:22 mypayload.txt


In [81]:
import boto3

client = boto3.client('sagemaker-runtime')

endpoint_name = "ct-scan-body-part-detector"
content_type = "application/json"
accept = "application/json"
payload = mypayload # Payload for inference.

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType=content_type,
    Accept=accept,
    Body=mypayloadstr
    )

print(response)
result = json.loads(response['Body'].read().decode())


{'ResponseMetadata': {'RequestId': '831933cd-b1db-4ab6-a6cd-c84a345c0eec', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '831933cd-b1db-4ab6-a6cd-c84a345c0eec', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sun, 12 Sep 2021 21:47:24 GMT', 'content-type': 'application/json', 'content-length': '422'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7fae2d243780>}
{'body_parts': {'abdomen': {'height': 54.225000858306885, 'unit': 'mm'}, 'chest': {'height': 210.8750033378601, 'unit': 'mm'}, 'crus': {'height': 0.0, 'unit': 'mm'}, 'foot': {'height': 0.0, 'unit': 'mm'}, 'head': {'height': 0.0, 'unit': 'mm'}, 'neck': {'height': 36.15000057220459, 'unit': 'mm'}, 'pelvis': {'height': 0.0, 'unit': 'mm'}, 'thigh': {'height': 0.0, 'unit': 'mm'}}, 'cap_completeness': {'abdomen': False, 'chest': True, 'pelvis': False}, 'head_first': True}


In [124]:
result

{'body_parts': {'abdomen': {'height': 54.225000858306885, 'unit': 'mm'},
  'chest': {'height': 210.8750033378601, 'unit': 'mm'},
  'crus': {'height': 0.0, 'unit': 'mm'},
  'foot': {'height': 0.0, 'unit': 'mm'},
  'head': {'height': 0.0, 'unit': 'mm'},
  'neck': {'height': 36.15000057220459, 'unit': 'mm'},
  'pelvis': {'height': 0.0, 'unit': 'mm'},
  'thigh': {'height': 0.0, 'unit': 'mm'}},
 'cap_completeness': {'abdomen': False, 'chest': True, 'pelvis': False},
 'head_first': True}

In [125]:
with open('response_output.json','w') as f:
    f.write(json.dumps(result))

In [82]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name ct-scan-body-part-detector \
    --body fileb://mypayload.txt \
    --content-type "application/json" \
    --region $sagemaker_session.boto_region_name \
    output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


In [83]:
!cat output.json

{"body_parts":{"abdomen":{"height":54.225000858306885,"unit":"mm"},"chest":{"height":210.8750033378601,"unit":"mm"},"crus":{"height":0.0,"unit":"mm"},"foot":{"height":0.0,"unit":"mm"},"head":{"height":0.0,"unit":"mm"},"neck":{"height":36.15000057220459,"unit":"mm"},"pelvis":{"height":0.0,"unit":"mm"},"thigh":{"height":0.0,"unit":"mm"}},"cap_completeness":{"abdomen":false,"chest":true,"pelvis":false},"head_first":true}


In [84]:
# delete endpoint

In [85]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

In [87]:
# unseen dataset from tcia 
# collection id # anatomical-site # series-isntance-uid
# OPC-Radiomics, # head-and-neck # '1.3.6.1.4.1.14519.5.2.1.8666.3098.135047974513108102532692126459'
# CPTAC-SAR # whole-body # '1.3.6.1.4.1.14519.5.2.1.3320.3273.193828570195012288011029757668'
# COVID-19-NY-SBU # abdomen-pelvis # '1.3.6.1.4.1.14519.5.2.1.99.1071.20813077754558159326970236991440'

series_instance_uid_list = [
    "1.3.6.1.4.1.14519.5.2.1.8666.3098.135047974513108102532692126459",
    "1.3.6.1.4.1.14519.5.2.1.3320.3273.193828570195012288011029757668",
    "1.3.6.1.4.1.14519.5.2.1.99.1071.20813077754558159326970236991440",
]

In [88]:
niigz_list = []
image_root = 'myimages'
os.makedirs(image_root,exist_ok=True)
for suid in series_instance_uid_list:
    nii_gz_path = os.path.join(image_root,f'{suid}.nii.gz')
    folder_path = os.path.join(image_root,suid)
    if not os.path.exists(folder_path):
        folder_path = download_images(suid,image_root=image_root)        
    if not os.path.exists(nii_gz_path):
        nii_gz_path = convert_to_nifti(folder_path,suid,image_root=image_root)
    myjson = convert_to_json(nii_gz_path)
    niigz_list.append(myjson)
    
batch_file = 'niigz-jsonlines.txt'
with open(batch_file,'w') as f:
    for x in niigz_list:
        f.write(json.dumps(x)+'\n')

200
200
200


In [91]:
# upload the batch-transform job input files to S3
transform_input_file = batch_file
transform_input = sagemaker_session.upload_data(transform_input_file, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-west-2-535328050074/ct-scan-body-part-detector/niigz-jsonlines.txt


In [96]:
transform_output_folder = "NEWbatch-transform-output"
output_path="s3://{}/{}".format(bucket, transform_output_folder)

In [97]:
# Run the batch-transform job
transformer = model.transformer(
    1,batch_transform_inference_instance_type,
    output_path=output_path,
    assemble_with='Line',
    accept='application/jsonlines',
)
transformer.transform(transform_input, content_type='application/jsonlines', split_type='Line')
transformer.wait()

.............................Starting the inference server with 4 workers.
[2021-09-12 22:10:05 +0000] [10] [INFO] Starting gunicorn 20.1.0
[2021-09-12 22:10:05 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2021-09-12 22:10:05 +0000] [10] [INFO] Using worker: sync
[2021-09-12 22:10:05 +0000] [14] [INFO] Booting worker with pid: 14
[2021-09-12 22:10:05 +0000] [15] [INFO] Booting worker with pid: 15
[2021-09-12 22:10:05 +0000] [17] [INFO] Booting worker with pid: 17
[2021-09-12 22:10:05 +0000] [18] [INFO] Booting worker with pid: 18
[Sun, 12 Sep 2021 22:10:07] DEBUG [tpu_cluster_resolver.py.<module>:35] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[Sun, 12 Sep 2021 22:10:07] DEBUG [tpu_cluster_resolver.py.<module>:35] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
[Sun, 12 Sep 2021 22:10:08] DEBUG [tpu_cluster_r

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0oda9g+SSRNKjdFtVardgx9H/yGjsa/m2HH0L/QAn7OOp6//Tq0vaxHCu9Rsor+hMXQAuKIt6sH1fxtrE73PPQuMLnj24lLjgvijyChkP3JvZl5DXhH/p7WFNk9aIC8pPbpp0kDLb/cHPdwfcoTcLPqcWIn7R+5/IRewt93Gw99HPswplb/d36Pud2GpPI7MBmesYezrlC6z/ytK5d9tjbawH30NSPTCcnsHZMrssu468ugKdNrsUv0YfFJ8BOxf5AE5auIlihl6TnGQHr6vuNxQ/I/XPrj3d8a/nWNb9CJ2n9fXup3lthl8i3zH/kf2su4mvN7ixMdtON9Tz82YEdYHZjwJdd9uMyzLc9gX7ksuN7qAZufnPfA8OEbWEZfevFS+xzYkRyW5P6VUe3HTUuvSBuHnvtLAfeH/QdeCe2pqsNfJjSHb3R7Hl4df8NvdfseW5h70udtjrB3O+BzwRzuO121Kfy3/stK/7vYy23Lc14xFJ+j8tvkMX6sXfQBdoN/QERzjW4llzixVhpGvxk+QDvXalGml1i0OsgfuL5yuz77zc0Ol5mS9vgJd7zrN47czst88zopNBm7pF3xs9TFre7e4XGrn4VXkg4/fAM9LSz9ft20ttja5l/WGvU3kvY81gW58PBF+rts0xBiRC8hMjxMRC0GvUldALlnjENdnA/yh9v9YcmVCqbU0U0uN13usCx2ObsPHIwbvfMwW2ek2GNeBH2x+cMkazC2N8B/7cKm15fgdbs4tK/30gJ5hbJGvP82a53wPcUe3daGVbUvVdfQRtg4+AWNX1E/KIRNrJE6oe4gTif+eU8a2ze8vm7Bt+5xSbVjo1uOwvgWQgfh6yErXCa7r8bmQC+63I6PBOYAeBg/L7T84h57AN7SxyK4DxpIR2Io802UP+mY7a9Njkdg4yxOXKxLHi0uNKWMbe42S1zrpemrjBsWK72/8tzSAv4rfM8veHZ8G2Y7uRj5i98LrPj4PnPs69/Tx

In [99]:
# output is available on following path
transformer.output_path

's3://sagemaker-us-west-2-535328050074/NEWbatch-transform-output'

In [116]:
s3_client = sagemaker_session.boto_session.client('s3')
local_path = f'{batch_file}.out'
store_path = f"{transform_output_folder}/niigz-jsonlines.txt.out"
#store_path = f"{transform_output_folder}/{transformer.latest_transform_job.name}/{batch_file}.out"
print(store_path)
s3_client.download_file(bucket, store_path, local_path)
with open(local_path) as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

NEWbatch-transform-output/niigz-jsonlines.txt.out
Transform results: 
{"body_parts": {"abdomen": {"height": 0.0, "unit": "mm"}, "chest": {"height": 59.75999879837036, "unit": "mm"}, "crus": {"height": 0.0, "unit": "mm"}, "foot": {"height": 0.0, "unit": "mm"}, "head": {"height": 165.9999966621399, "unit": "mm"}, "neck": {"height": 106.23999786376953, "unit": "mm"}, "pelvis": {"height": 0.0, "unit": "mm"}, "thigh": {"height": 0.0, "unit": "mm"}}, "cap_completeness": {"abdomen": false, "chest": false, "pelvis": false}, "head_first": true}
{"body_parts": {"abdomen": {"height": 390.0, "unit": "mm"}, "chest": {"height": 156.0, "unit": "mm"}, "crus": {"height": 0.0, "unit": "mm"}, "foot": {"height": 897.0, "unit": "mm"}, "head": {"height": 195.0, "unit": "mm"}, "neck": {"height": 78.0, "unit": "mm"}, "pelvis": {"height": 117.0, "unit": "mm"}, "thigh": {"height": 117.0, "unit": "mm"}}, "cap_completeness": {"abdomen": true, "chest": true, "pelvis": true}, "head_first": true}
{"body_parts": {"ab

In [104]:
model.delete_model()

In [117]:
!cat niigz-jsonlines.txt.out

{"body_parts": {"abdomen": {"height": 0.0, "unit": "mm"}, "chest": {"height": 59.75999879837036, "unit": "mm"}, "crus": {"height": 0.0, "unit": "mm"}, "foot": {"height": 0.0, "unit": "mm"}, "head": {"height": 165.9999966621399, "unit": "mm"}, "neck": {"height": 106.23999786376953, "unit": "mm"}, "pelvis": {"height": 0.0, "unit": "mm"}, "thigh": {"height": 0.0, "unit": "mm"}}, "cap_completeness": {"abdomen": false, "chest": false, "pelvis": false}, "head_first": true}
{"body_parts": {"abdomen": {"height": 390.0, "unit": "mm"}, "chest": {"height": 156.0, "unit": "mm"}, "crus": {"height": 0.0, "unit": "mm"}, "foot": {"height": 897.0, "unit": "mm"}, "head": {"height": 195.0, "unit": "mm"}, "neck": {"height": 78.0, "unit": "mm"}, "pelvis": {"height": 117.0, "unit": "mm"}, "thigh": {"height": 117.0, "unit": "mm"}}, "cap_completeness": {"abdomen": true, "chest": true, "pelvis": true}, "head_first": true}
{"body_parts": {"abdomen": {"height": 170.34000778198242, "unit": "mm"}, "chest": {"heigh